In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

fileDay = datetime.date.today().strftime('%Y%m%d')
fileYear = datetime.date.today().strftime('%Y')

country = []
totCase = []

nationLink = [] # www.worldometers.info/coronavirus/ + 'country/name'

def scrapingWorldometers():
    url = "https://www.worldometers.info/coronavirus/"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    colList = []
    dateList = []
    caseDataList = []
    deathDataList = []
    df = None
    
    #if yda:
        #table = soup.select_one('table#main_table_countries_yesterday')
    #else:    
    table = soup.select_one('table#main_table_countries_today')
    #pprint(table)

    columns = table.select('tr')
    isCol = True

    for col in columns:
        if isCol:
            cols = col.select('th')
            for c in cols:
                colList.append(c.text)
            isCol = False
        else:
            rows = col.select('td') # '#' [0] column was added, so that index changed in 5. 15.
            cases = int(rows[2].text.strip().replace(',',''))
            if (cases >= 1000) and rows[1].a: 
                country.append(rows[1].text.strip())
                nationLink.append(rows[1].a["href"])
                totCase.append(cases)
            else:
                continue
                
    # Réunion, Curaçao => Reunion, Curacao
    if "Réunion" in country:
        idx = country.index("Réunion")
        country[idx] = "Reunion"
    
    if "Curaçao" in country:
        idx = country.index("Curaçao")
        country[idx] = "Curacao"

            
    for link in nationLink:
        html = requests.get(url + link).text
        soup = BeautifulSoup(html, 'html.parser')
        jsList = soup.find_all('script', attrs={'type' : "text/javascript"})
        for js in jsList:
            if js.text.find("Highcharts") != -1:
                if js.text.find("'coronavirus-cases-linear'") != -1:
                    caseScript = js.text
                    frontCut = caseScript[caseScript.find("categories:"):]
                    caseDaysStartIdx = frontCut.find("[") + 1 # start from inside of []
                    caseDaysEndIdx = frontCut.find("]")
                    caseDaysList = frontCut[caseDaysStartIdx:caseDaysEndIdx].split(",")
                    
                    startDate = datetime.datetime.strptime(caseDaysList[0].strip('"') + ' 2020', "%b %d %Y")
                    endDate = datetime.datetime.strptime(caseDaysList[-1].strip('"') + ' ' + fileYear, "%b %d %Y")
                    dateList = pd.date_range(startDate, endDate)
                    
                    frontCut = frontCut[frontCut.find("data:"):]
                    caseDataStartIdx = frontCut.find("[") + 1 # start from inside of []
                    caseDataEndIdx = frontCut.find("]")
                    caseDataList = frontCut[caseDataStartIdx:caseDataEndIdx].split(",")
                    
                elif js.text.find("coronavirus-deaths-linear") != -1:
                    deathScript = js.text
                    frontCut = deathScript[deathScript.find("data:"):]
                    deathDataStartIdx = frontCut.find("[") + 1 # start from inside of []
                    deathDataEndIdx = frontCut.find("]")
                    deathDataList = frontCut[deathDataStartIdx:deathDataEndIdx].split(",")
        
        nationDict = {"Country": country[nationLink.index(link)], "Date":dateList, "TotalCases":caseDataList, "TotalDeaths":deathDataList}
        dfByNation = pd.DataFrame(nationDict)
        df = pd.concat([df, dfByNation])
        
    df["Date"] = df["Date"].astype(str) # datetime64[ns] -> str
    df = df.set_index(["Country", "Date"])
    return df

In [2]:
def toCSV(df):
    csvFile = './covidDataset/' + fileDay + 'CoronaWorld.csv' # set directory properly
    df.to_csv(csvFile)
    print(fileDay, "today csv complete")
    
def toJSON(df, orient='split'):
    jsonFile = './covidDataset/' + fileDay + 'CoronaWorld.json'
    df.to_json(jsonFile, orient)
    print(fileDay, "today json complete")

In [3]:
covidDataFrame = scrapingWorldometers()
print(covidDataFrame) # only china start from 2020-01-22

                   TotalCases TotalDeaths
Country Date                             
USA     2020-02-15         15           0
        2020-02-16         15           0
        2020-02-17         15           0
        2020-02-18         15           0
        2020-02-19         15           0
...                       ...         ...
China   2020-05-10      82901        4633
        2020-05-11      82918        4633
        2020-05-12      82919        4633
        2020-05-13      82926        4633
        2020-05-14      82929        4633

[8934 rows x 2 columns]


In [4]:
toCSV(covidDataFrame)
toJSON(covidDataFrame)

20200515 today csv complete
20200515 today json complete
